In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional, Conv2D, MaxPooling2D, Flatten, LeakyReLU, AveragePooling2D
from keras.utils import to_categorical
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import glob
from sklearn.utils import shuffle
from sklearn import metrics
from io import StringIO
import warnings
import gc
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import roc_auc_score
import random
import seaborn as sns

In [2]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [3]:
from keras import backend as K

def auc_filter(y_true, y_pred):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=1)
    a1 = metrics.auc(fpr, tpr)
    
    return a1

def auc_print(y_true, y_pred):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=1)
    a1 = metrics.auc(fpr, tpr)
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=0)
    a0 = metrics.auc(fpr, tpr)
    
    return a1, a0

In [4]:
warnings.filterwarnings("ignore")

patience = 25
if os.path.exists("../input/birdsong-recognition/test_audio/"):
    n_epochs=50
    fc=100
else:
    n_epochs=50
    fc=100

prob_cutoff = 0.5

prepared_test_str = """row_id,site,audio_id,seconds
bulori/XC128942,site_3,bulori/XC128942,
bulori/XC170988,bulori/XC170988,bulori/XC170988,5
normoc/XC54018,normoc/XC54018,normoc/XC54018,5
normoc/XC62791,normoc/XC62791,normoc/XC62791,5
herthr/XC53784,herthr/XC53784,herthr/XC53784,5
herthr/XC119596,herthr/XC119596,herthr/XC119596,5
brnthr/XC31308,brnthr/XC31308,brnthr/XC31308,5
brnthr/XC53695,brnthr/XC53695,brnthr/XC53695,5
vesspa/XC17095,vesspa/XC17095,vesspa/XC17095,5
vesspa/XC17096,vesspa/XC17096,vesspa/XC17096,5
solsan/XC17025,solsan/XC17025,solsan/XC17025,5
norfli/XC11578,norfli/XC11578,norfli/XC11578,5
lesnig/XC27724,lesnig/XC27724,lesnig/XC27724,5
grycat/XC31058,grycat/XC31058,grycat/XC31058,5
eastow/XC53188,eastow/XC53188,eastow/XC53188,5
aldfly/XC2628,aldfly/XC2628,aldfly/XC2628,5
ameavo/XC99571,ameavo/XC99571,ameavo/XC99571,5
amebit/XC127371,amebit/XC127371,amebit/XC127371,5
amecro/XC51410,amecro/XC51410,amecro/XC51410,5
amegfi/XC17120,amegfi/XC17120,amegfi/XC17120,5
"""

# data parameters
n_birds = 5
mfcc_directory = '../input/mfcc-50/'
train_dir = '../input/birdsong-recognition/train_audio/'
birds = os.listdir(train_dir)
random.shuffle(birds)

birdHalf1 = birds[:int(len(birds)/2)]
birdHalf2 = birds[int(len(birds)/2):]
# birds = birds[:n_birds]
if os.path.exists("../input/birdsong-recognition/test_audio/"):
#     birds = ["treswa", "bulori", "herthr", "brnthr", "vesspa"]
    neglist = ["reshaw","grbher3","rthhum","pinjay","rocpig","yebfly","foxspa","casfin","gadwal","belspa2","whbnut","bkcchi","brespa","aldfly","bktspa","houfin","labwoo","reevir1","dowwoo","wewpew","pygnut","houwre","bkhgro","sposan","caster1","winwre3","sposan","labwoo","moudov","gnwtea","houwre","dusfly","normoc","boboli","gadwal","wewpew","winwre3","marwre","dowwoo","wewpew","vigswa","caster1","comgol","coohaw","wesmea","killde","wessan","dusfly","amepip","rufhum","gnwtea","marwre","evegro","casfin","westan","belspa2","blujay","olsfly","amecro","casfin","sposan","linspa","olsfly","grbher3","blugrb1","stejay","savspa","wilfly","hoowar","killde","winwre3","mallar3","fiespa","reshaw","lazbun","linspa","rebwoo","bktspa","horlar","magwar","bnhcow","cowscj1","scoori","rocpig","bkcchi","reevir1","spotow","belspa2","belkin1","blugrb1","pasfly","amepip","dowwoo","sposan","pinsis","cedwax","westan","clanut","rufhum","blugrb1"]
    birds = list(set(birds)-set(neglist))[0:10]
else:
    birds = ["treswa"]
#     birds = ['grhowl', 'treswa', 'tunswa', 'yebsap', 'norpar', 'rebsap', 'commer', 'redcro', 'leabit', 'lesgol']
#     neglist = ["reshaw","grbher3","rthhum","pinjay","rocpig","yebfly","foxspa","casfin","gadwal","belspa2","whbnut","bkcchi","brespa","aldfly","bktspa","houfin","labwoo","reevir1","dowwoo","wewpew","pygnut","houwre","bkhgro","sposan","caster1","winwre3","sposan","labwoo","moudov","gnwtea","houwre","dusfly","normoc","boboli","gadwal","wewpew","winwre3","marwre","dowwoo","wewpew","vigswa","caster1","comgol","coohaw","wesmea","killde","wessan","dusfly","amepip","rufhum","gnwtea","marwre","evegro","casfin","westan","belspa2","blujay","olsfly","amecro","casfin","sposan","linspa","olsfly","grbher3","blugrb1","stejay","savspa","wilfly","hoowar","killde","winwre3","mallar3","fiespa","reshaw","lazbun","linspa","rebwoo","bktspa","horlar","magwar","bnhcow","cowscj1","scoori","rocpig","bkcchi","reevir1","spotow","belspa2","belkin1","blugrb1","pasfly","amepip","dowwoo","sposan","pinsis","cedwax","westan","clanut","rufhum","blugrb1"]
#     birds = list(set(birds)-set(neglist))[0:10]

trainTable = pd.read_csv("../input/birdsong-recognition/train.csv")
trainTable1 = trainTable[trainTable["ebird_code"].isin(birdHalf1)]
# trainTable2 = trainTable[trainTable["ebird_code"].isin(['pinwar','plsvir','prawar','purfin','pygnut','rebmer','rebnut','rebsap','rebwoo'])]
trainTable2 = trainTable[trainTable["ebird_code"].isin(birdHalf2)]

neg2 = list(trainTable1.agg(lambda x: train_dir + x.ebird_code + "/" + x.filename, axis=1).sample(n=100))
neg1 = list(trainTable2.agg(lambda x: train_dir + x.ebird_code + "/" + x.filename, axis=1).sample(n=100))

In [5]:
birds

['treswa']

In [6]:
def load_test_audio(path):
    signal, blah = librosa.load(path, sr=22050)
    if len(signal.shape)==1:
        y = signal
    else:
        y = np.average(signal, axis=1) 
    return signal
    

def getBirdMfcc(bird, n_mfcc=50, filecount=None):
    birddirectory = train_dir + "/" + bird
    birdfiles = [os.path.join(train_dir, bird, birdfile) for birdfile in os.listdir(birddirectory)]
    return getFilesMfcc(birdfiles, n_mfcc=n_mfcc, filecount=filecount)

def getBirdMfccPickle(bird, n_mfcc=50, filecount=None):
    birddirectory = mfcc_directory + bird + ".pickle"
    mfcc = pickle.load(open(birddirectory,'rb')).T
    return mfcc

def getFilesMfcc(files, n_mfcc=50, filecount=None):
    if filecount!=None:
        files = files[:filecount]
    
    mfcc = []
    for file in files:
        print("reading file: {}".format(file))
        try:
            new_y = load_test_audio(file)
        except:
            continue
        if len(new_y)==0:
            continue
        new_mfcc = librosa.feature.mfcc(n_mfcc=50, y=new_y, sr=22050)
        mfcc.append(new_mfcc)
        del(new_y)
    gc.collect()
    mfcc = np.concatenate(mfcc,axis=1).T
    return mfcc

In [7]:
dataMap = {}
testMap = {}

# birds mfcc calcualtion
for bird in birds:
    bird_mfcc = getBirdMfccPickle(bird, filecount=fc)
    train_samples = int(0.9*bird_mfcc.shape[0])
    test_samples = -1*int(0.1*bird_mfcc.shape[0])
    dataMap[bird] = bird_mfcc[:train_samples,]
    testMap[bird] = bird_mfcc[test_samples:,]

# neg mfcc calculation
bird_mfcc = getBirdMfccPickle('neg1', filecount=100).T
train_samples = int(0.9*bird_mfcc.shape[0])
test_samples = -1*int(0.1*bird_mfcc.shape[0])
dataMap["neg1"] = bird_mfcc[:train_samples,]
testMap["neg1"] = bird_mfcc[test_samples:,]

histories = {}
models = {}

In [8]:
def train_generator(bird):
    while True:
        sequence_length = 215
        
        xarr = []
        yarr = []
        
        for k,v in {bird:dataMap[bird], "neg1":dataMap["neg1"]}.items():
            nsamples = v.shape[0]
            size=3
            samples = np.random.randint(0,nsamples-sequence_length+1,size=size)
            if k==bird:
                birdohe = np.array([[1]])
            else:
                birdohe = np.array([[0]])
            for sample in samples:
                xf = v[sample:sample+sequence_length,:]
                lb, ub = np.percentile(xf, 5), xf.max()
                xp, yp = [xf < lb, xf > ub , np.logical_not(np.logical_and(xf < lb , xf > ub))], [0, 1, lambda x: (x-lb)/ub]
                xt = np.piecewise(xf, xp, yp)
                
                xarr.append(xt.reshape(1,sequence_length,50,1))
                yarr.append(birdohe.reshape(1,1))
        x_train = np.concatenate(xarr)
        y_train = np.concatenate(yarr)
        x_train, y_train = shuffle(x_train, y_train)
        yield x_train, y_train

def test_generator(bird, size=1500):
    sequence_length = 215

    xarr = []
    yarr = []

    for k,v in {bird:testMap[bird], "neg1":testMap["neg1"]}.items():
        nsamples = v.shape[0]
        samples = np.random.randint(0,nsamples-sequence_length+1,size=size)
        if k==bird:
            birdohe = np.array([[1]])
        else:
            birdohe = np.array([[0]])
        for sample in samples:
            xf = v[sample:sample+sequence_length,:]
            lb, ub = np.percentile(xf, 5), xf.max()
            xp, yp = [xf < lb, xf > ub , np.logical_not(np.logical_and(xf < lb , xf > ub))], [0, 1, lambda x: (x-lb)/ub]
            xt = np.piecewise(xf, xp, yp)

            xarr.append(xt.reshape(1,sequence_length,50,1))
            yarr.append(birdohe.reshape(1,1))
    x_train = np.concatenate(xarr)
    y_train = np.concatenate(yarr)
    x_train, y_train = shuffle(x_train, y_train)
    return x_train, y_train

def sample_splitter(v):
    sequence_length = 215
    
    rowsl = v.shape[0]
    nsamples = int(rowsl/sequence_length)
    
    xarr = []
    for i in range(nsamples):
        sample = i*sequence_length
        xf = v[sample:sample+sequence_length,:]
        lb, ub = np.percentile(xf, 5), xf.max()
        xp, yp = [xf < lb, xf > ub , np.logical_not(np.logical_and(xf < lb , xf > ub))], [0, 1, lambda x: (x-lb)/ub]
        xt = np.piecewise(xf, xp, yp)
        xarr.append(xt.reshape(1,sequence_length,50,1))
    
    if rowsl-25>nsamples*sequence_length:
        sample = nsamples*sequence_length
        xf = v[sample:sample+sequence_length,:]
        lb, ub = np.percentile(xf, 5), xf.max()
        xp, yp = [xf < lb, xf > ub , np.logical_not(np.logical_and(xf < lb , xf > ub))], [0, 1, lambda x: (x-lb)/ub]
        xt = np.piecewise(xf, xp, yp)
        
        mfccdim = xt.shape
        print(mfccdim)
        diff = sequence_length - mfccdim[0]
        lastone = xt[mfccdim[0]-1,:]
        xt = np.vstack([xt, np.tile(lastone, (diff, 1))])
        xarr.append(xt.reshape(1,sequence_length,50,1))
    
    x_train = np.concatenate(xarr)
    
    return x_train

In [9]:
def get_model(): 
#     with tpu_strategy.scope():
    model = Sequential()

    model.add(Conv2D(4, kernel_size=(5, 1), activation='tanh', input_shape=(215, 50, 1)))
    model.add(MaxPooling2D((3, 1), padding='same'))
    model.add(Conv2D(2, kernel_size=(3, 3), activation='tanh'))
    model.add(MaxPooling2D((8,1), strides=(8,1), padding='same'))
    model.add(Flatten())
    model.add(keras.layers.Dropout(0.9))
    model.add(Dense(1, activation='sigmoid'))

#     for layer in model.layers:
#         print(layer.output_shape)
#         if "conv" in layer.name:
#             filters, biases = layer.get_weights()
#             print(layer.name, filters.shape)

    adamo=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)
    model.compile(loss='binary_crossentropy',
                  optimizer=adamo,
                  metrics=[keras.metrics.AUC(name="auc")])

    gc.collect()
    return model

In [10]:
for bird in birds:
    x_val,y_val = test_generator(bird, size=3000)
    
    model = get_model()
    print("Modeling bird: {}".format(bird))
    my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='auc', patience=4, mode="max", restore_best_weights=True)]
    
    history = model.fit_generator(train_generator(bird), 
                                  steps_per_epoch=400, 
                                  epochs=40, verbose=1, 
                                  validation_data=(x_val, y_val),
                                  callbacks=my_callbacks)
    
    models[bird] = model
    histories[bird] = history
    del(history)
    del(model)

Modeling bird: treswa
Epoch 1/40
400/400 [==============================] - 15s 36ms/step - loss: 0.7276 - auc: 0.6088 - val_loss: 0.7499 - val_auc: 0.4832
Epoch 2/40
400/400 [==============================] - 14s 36ms/step - loss: 0.6328 - auc: 0.7071 - val_loss: 0.7198 - val_auc: 0.5992
Epoch 3/40
400/400 [==============================] - 14s 35ms/step - loss: 0.6009 - auc: 0.7451 - val_loss: 0.6771 - val_auc: 0.6546
Epoch 4/40
400/400 [==============================] - 15s 36ms/step - loss: 0.5835 - auc: 0.7617 - val_loss: 0.7279 - val_auc: 0.6294
Epoch 5/40
400/400 [==============================] - 14s 35ms/step - loss: 0.5611 - auc: 0.7866 - val_loss: 0.7282 - val_auc: 0.6497
Epoch 6/40
400/400 [==============================] - 14s 35ms/step - loss: 0.5482 - auc: 0.7997 - val_loss: 0.5531 - val_auc: 0.8102
Epoch 7/40
400/400 [==============================] - 14s 36ms/step - loss: 0.5249 - auc: 0.8164 - val_loss: 0.6440 - val_auc: 0.7172
Epoch 8/40
400/400 [====================

In [11]:
# for bird in birds:
#     print("bird report: {} --------------------------------------------------".format(bird))
    
#     model = models[bird]
#     x_val,y_val = test_generator(bird,size=5000)
#     preds = model.predict(x_val)
#     print("validation loss: {}".format(model.evaluate(x_val, y_val, verbose=False)))
#     print("validation  auc: {}".format(auc_print(y_val, preds)))
#     preds[preds > 0.5] = 1
#     preds[preds <= 0.5] = 0
#     [print("{}: {}".format(m,e)) for e,m in zip(prfs(y_val,preds,labels=[1,0]),["precision","   recall","  f-score","   counts"])]
    
#     print("------------------------------------------------------------------".format(bird))
    
# for bird in birds:
#     history = histories[bird]
#     plt.subplot(2,1,1)
#     plt.plot(history.history["loss"])
#     plt.plot(history.history["val_loss"])

#     plt.subplot(2,1,2)
#     plt.plot(history.history["auc"])
#     plt.plot(history.history["val_auc"])
#     title_obj = plt.title(bird)
# #     plt.getp(title_obj)                    #print out the properties of title
# #     plt.getp(title_obj, 'text')            #print out the 'text' property for title
# #     plt.setp(title_obj, color='r')
#     plt.show()
#     plt.clf()

Model refining

In [12]:
refinedModels = {}

refinedWriter = open("refined.txt","a")

for mbird,bmodel in models.items():
    x_val,y_val = test_generator(mbird,size=5000)
    preds = bmodel.predict(x_val)
    af = auc_filter(y_val, preds)
    print(af)
    if(af>0.75):
        refinedModels[mbird] = bmodel
    refinedWriter.write("{}:{}\n".format(mbird,af))

refinedWriter.close()

print("Refined model count: {}".format(len(refinedModels.keys())))

0.82121044
Refined model count: 1


In [13]:
if os.path.exists("../input/birdsong-recognition/test_audio/"):
    test_dir = "../input/birdsong-recognition/test_audio/"
    testtable = pd.read_csv("../input/birdsong-recognition/test.csv")
else:
    test_dir = "../input/birdsong-recognition/train_audio/"
    testtable = pd.read_csv(StringIO(prepared_test_str))


def load_test_clip(signal, start_time, duration=5):
    sr=22050
    maxl = len(signal)
    
    if duration!=None:
        y = signal[max(0,int(start_time)*22050):min(int(start_time+duration)*22050,maxl)]
    else:
        y = signal[max(0,start_time*22050):]
    
    mfcc_feat = librosa.feature.mfcc(n_mfcc=50, y=y, sr=22050)
    return mfcc_feat.T

def make_predictions(sound_clip):
    sound_x = sample_splitter(sound_clip)
    
    predlist = []
    for mbird,bmodel in refinedModels.items():
        sound_y = bmodel.predict(sound_x)
        prob = make_probability(sound_x, bmodel)
        if prob>0.75:
            predlist.append(mbird)
    
    if len(predlist)==0:
        predbirds = "nocall"
    else:
        predbirds = " ".join(predlist)
    
    return predbirds
    
# // deprecated
def make_probability(sound_x, bmodel):
    sound_y = bmodel.predict(sound_x)
    
    prob = sound_y.max()
    return prob

def reshape_pad_sound(sound_clip):
    mfccdim = sound_clip.shape
    diff = 215 - mfccdim[0]
    if diff!=0:
        lastone = sound_clip[mfccdim[0]-1,:]
        sound_clip = np.vstack([sound_clip, np.tile(lastone, (diff, 1))])
    sound_x = sound_clip.reshape(1,215,mfccdim[1], 1)
    
    return sound_x
    

In [14]:
c = np.array([[1,2,3,4]])
np.tile(c,(4,1))

array([[1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4],
       [1, 2, 3, 4]])

In [15]:
test_info = testtable.sort_values("audio_id")

preds = []
sound_read = ""
for index, row in test_info.iterrows():
    # Get test row information
    site = row['site']
    try:
        start_time = row['seconds'] - 5
    except:
        start_time = 0
    row_id = row['row_id']
    audio_id = row['audio_id']
    print(audio_id)

    try:
        if sound_read != audio_id:
            sound_audio = load_test_audio(test_dir + audio_id + '.mp3')
            sound_read = audio_id
    except:
        print("exception")
        pred = "nocall"
        preds.append(pred)
        continue
        
    if site == 'site_3':
        sound_clip = load_test_clip(sound_audio, 0, duration=None)
    else:
        sound_clip = load_test_clip(sound_audio, start_time)
            
#     try:
#         if len(refinedModels.items)==0:
#             pred = "aldfly"
#         else:
    pred = make_predictions(sound_clip)
#     except:
#         pred = "nocall"
    preds.append(pred)
    
test_info['birds'] = preds
testtable = test_info.sort_index()

test_submission = testtable.drop(['site', 'audio_id', 'seconds'], axis = 1)

test_submission.to_csv('submission.csv', index = None)

aldfly/XC2628
ameavo/XC99571
amebit/XC127371
amecro/XC51410
amegfi/XC17120
brnthr/XC31308
brnthr/XC53695
bulori/XC128942
(208, 50)
bulori/XC170988
eastow/XC53188
grycat/XC31058
herthr/XC119596
herthr/XC53784
lesnig/XC27724
norfli/XC11578
normoc/XC54018
normoc/XC62791
solsan/XC17025
(147, 50)
vesspa/XC17095
vesspa/XC17096


In [16]:
print(test_submission.head(20))

             row_id   birds
0   bulori/XC128942  nocall
1   bulori/XC170988  nocall
2    normoc/XC54018  nocall
3    normoc/XC62791  nocall
4    herthr/XC53784  nocall
5   herthr/XC119596  nocall
6    brnthr/XC31308  nocall
7    brnthr/XC53695  nocall
8    vesspa/XC17095  nocall
9    vesspa/XC17096  nocall
10   solsan/XC17025  nocall
11   norfli/XC11578  nocall
12   lesnig/XC27724  nocall
13   grycat/XC31058  nocall
14   eastow/XC53188  nocall
15    aldfly/XC2628  nocall
16   ameavo/XC99571  nocall
17  amebit/XC127371  nocall
18   amecro/XC51410  nocall
19   amegfi/XC17120  nocall


In [17]:
# # single bird call
# bird = birds[0]
# x_val,y_val = test_generator(bird)

# model = get_model()
# my_callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=25, mode="max", restore_best_weights=True)]
    
# history = model.fit_generator(train_generator(bird), steps_per_epoch=150, epochs=100, verbose=1, validation_data=(x_val, y_val))

# plt.subplot(2,1,1)
# plt.plot(history.history["loss"])
# plt.plot(history.history["val_loss"])

# plt.subplot(2,1,2)
# plt.plot(history.history["accuracy"])
# plt.plot(history.history["val_accuracy"])
# plt.show()

# x_val,y_val = test_generator(bird)
# preds = model.predict(x_val)
# preds[preds > 0.5] = 1
# preds[preds <= 0.5] = 0

# prfs(y_val,preds,labels=[1,0])